In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import gc

import ydata_profiling as pdp

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df_train = pd.read_csv('train.csv')
print(df_train.shape)
df_train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [3]:
#ベースライン作成
#目的変数と説明変数の設定
x_train, y_train, id_train = df_train[['Pclass', 'Fare']], df_train[['Survived']], df_train[['PassengerId']]
id_train.head()

,PassengerId
0,1
1,2
2,3
3,4
4,5


In [17]:
#ホールドアウト検証
x_tr, x_va, y_tr, y_va = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, stratify=y_train, random_state=123)
print(x_tr.shape, y_tr.shape)
print(x_va.shape, y_va.shape)
print('y_train:{:.3f}, y_tr:{:.3f}, y_va:{:.3f}'.format(
    y_train['Survived'].mean(),
    y_tr['Survived'].mean(),
    y_va['Survived'].mean(),
))

(712, 2) (712, 1)
(179, 2) (179, 1)
y_train:0.384, y_tr:0.383, y_va:0.385


In [20]:
#ハイパーパラメータ

params = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : 'auc',
    'learning_rate' : 0.1,
    'num_leaves' : 16,
    'n_estimators' : 100000,
    'random_state' : 123,
    'importance_type' : 'gain',
}

model = lgb.LGBMClassifier(**params)
model.fit(x_tr,
          y_tr,
          eval_set=[(x_tr, y_tr), (x_va, y_va)],
          callbacks=[lgb.early_stopping(stopping_rounds=100)],
         )

[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000077 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 120
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383427 -> initscore=-0.475028
[LightGBM] [Info] Start training from score -0.475028
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[35]	training's auc: 0.81342	valid_1's auc: 0.761265


LGBMClassifier(importance_type='gain', metric='auc', n_estimators=100000,
               num_leaves=16, objective='binary', random_state=123)

In [23]:
y_tr_pred = model.predict(x_tr)
y_va_pred = model.predict(x_va)
metric_tr = accuracy_score(y_tr, y_tr_pred)
metric_va = accuracy_score(y_va, y_va_pred)
print(f'tr:{metric_tr},va:{metric_va}')

tr:0.7514044943820225,va:0.7262569832402235


In [25]:
imp = pd.DataFrame({'col':x_train.columns, 'imp':model.feature_importances_, 'nfold':nfold})
imp.sort_values('imp',ascending=False, ignore_index=True)

,col,imp
0,Fare,903.440373
1,Pclass,229.457186


In [16]:
params = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : 'auc',
    'learning_rate' : 0.2,
    'num_leaves' : 16,
    'n_estimators' : 100000,
    'random_state' : 123,
    'importance_type' : 'gain',
    'bagging_freq' : 1,
    'seed' : 123,
}



#クロスバリデーション

def train_cv(params):
    metrics = []
    imp = pd.DataFrame()

    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))

    for nfold in np.arange(5):
        print('-'*20,nfold,'-'*20)
        idx_tr, idx_va = cv[nfold][0],cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr,:], y_train.loc[idx_tr,:]
        x_va, y_va = x_train.loc[idx_va,:], y_train.loc[idx_va,:]
        print(x_tr.shape, y_tr.shape)
        print(x_va.shape, y_va.shape)
        print('y_train:{:.3f}, y_tr:{:.3f}, y_va:{:.3f}'.format(
            y_train['Survived'].mean(),
            y_tr['Survived'].mean(),
            y_va['Survived'].mean(),
            ))   

        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr,
              y_tr,
              eval_set=[(x_tr, y_tr), (x_va, y_va)],
              callbacks=[lgb.early_stopping(stopping_rounds=100)],
             )
        
        y_tr_pred = model.predict(x_tr)
        y_va_pred = model.predict(x_va)
        metric_tr = accuracy_score(y_tr, y_tr_pred)
        metric_va = accuracy_score(y_va, y_va_pred)
        print(f'tr:{metric_tr},va:{metric_va}')
        metrics.append([nfold, metric_tr, metric_va])
        
        _imp = pd.DataFrame({'col':input_x.columns, 'imp':model.feature_importances_, 'nfold':nfold})
        imp = pd.concat([imp,_imp], axis=0, ignore_index=True)

    print('-'*20,'result','-'*20)
    metrics = np.array(metrics)
    print(metrics)

    print('[cv]tr:{:.2f}+-{:.2f}, va:{:.2f}+-{:.2f}'.format(
        metrics[:,1].mean(), metrics[:,1].std(),
        metrics[:,2].mean(), metrics[:,2].std(),
    ))

    imp = imp.groupby('col')['imp'].agg(['mean','std'])
    imp.columns = ['imp', 'imp_std']
    imp = imp.reset_index(drop=False)

    print('Done')

    return imp, metrics

In [7]:
imp, metrics  = train_cv(x_train,y_train,id_train,params,n_splits=5)

-------------------- 0 --------------------
(712, 2) (712, 1)
(179, 2) (179, 1)
y_train:0.384, y_tr:0.383, y_va:0.385
[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000137 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 123
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383427 -> initscore=-0.475028
[LightGBM] [Info] Start training from score -0.475028
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[12]	training's auc: 0.793779	valid_1's auc: 0.740382
tr:0.7205056179775281,va:0.6759776536312849
-------------------- 1 --------------------
(713, 2) (713, 1)
(178, 2) (178, 1)
y_train:0.384, y_tr:0.384, y_va:0.382
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info

In [17]:
x_train, y_train, id_train = df_train[['Pclass', 'Fare','Age']], df_train[['Survived']], df_train[['PassengerId']]
print(x_train.shape, y_train.shape, id_train.shape)

(891, 3) (891, 1) (891, 1)


In [18]:
imp, metrics  = train_cv(x_train,y_train,id_train,params,n_splits=5)

-------------------- 0 --------------------
(712, 3) (712, 1)
(179, 3) (179, 1)
y_train:0.384, y_tr:0.383, y_va:0.385
[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000069 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 185
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383427 -> initscore=-0.475028
[LightGBM] [Info] Start training from score -0.475028
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[69]	training's auc: 0.913235	valid_1's auc: 0.72892
tr:0.824438202247191,va:0.7039106145251397
-------------------- 1 --------------------
(713, 3) (713, 1)
(178, 3) (178, 1)
y_train:0.384, y_tr:0.384, y_va:0.382
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] 

In [20]:
imp.sort_values('imp', ascending=False, ignore_index=True)

,col,imp,imp_std
0,Fare,547.621958,270.958097
1,Age,436.497719,247.807858
2,Pclass,299.843845,103.344422


In [23]:
#データの確認
df_train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PassengerId,891.0,NaN,NaN,NaN,446.0,257.353842,1.0,223.5,446.0,668.5,891.0
Survived,891.0,NaN,NaN,NaN,0.383838,0.486592,0.0,0.0,0.0,1.0,1.0
Pclass,891.0,NaN,NaN,NaN,2.308642,0.836071,1.0,2.0,3.0,3.0,3.0
Name,891,891,"Braund, Mr. Owen Harris",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,891,2,male,577,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,714.0,NaN,NaN,NaN,29.699118,14.526497,0.42,20.125,28.0,38.0,80.0
SibSp,891.0,NaN,NaN,NaN,0.523008,1.102743,0.0,0.0,0.0,1.0,8.0
Parch,891.0,NaN,NaN,NaN,0.381594,0.806057,0.0,0.0,0.0,0.0,6.0
Ticket,891,681,347082,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,891.0,NaN,NaN,NaN,32.204208,49.693429,0.0,7.9104,14.4542,31.0,512.3292


In [ ]:
pdp.ProfileReport(df_train)

In [31]:
#欠損値の補完 平均値
df_train['Age']=df_train['Age'].fillna(df_train['Age'].mean())
df_train['Age'].isnull().sum()

0

In [32]:
#欠損値の補完　空白埋め
df_train['Cabin']=df_train['Cabin'].fillna('')
df_train['Embarked']=df_train['Embarked'].fillna('')
df_train['Cabin'].isnull().sum()

0

In [34]:
#標準化
std = StandardScaler()
std.fit(df_train[['Fare']])
df_train['Fare_standard']=std.transform(df_train[['Fare']])
df_train[['Fare', 'Fare_standard']].head()

,Fare,Fare_standard
0,7.2500,-0.502445
1,71.2833,0.786845
2,7.9250,-0.488854
3,53.1000,0.420730
4,8.0500,-0.486337


In [36]:
#label encoding
le_embarked = LabelEncoder()
le_embarked.fit(df_train['Embarked'])
df_train['Embarked_le'] = le_embarked.transform(df_train['Embarked'])
df_train[['Embarked','Embarked_le']].head()

,Embarked,Embarked_le
0,S,2
1,C,0
2,S,2
3,S,2
4,S,2


In [37]:
df_train['Family'] = df_train['SibSp'] + df_train['Parch']
df_train[['SibSp','Parch', 'Family']].head()

,SibSp,Parch,Family
0,1,0,1
1,1,0,1
2,0,0,0
3,1,0,1
4,0,0,0


In [39]:
#性別ごとの運賃
df_train['mean_Fare_by_Sex'] = df_train.groupby('Sex')['Fare'].transform('mean')
df_train[['Sex', 'mean_Fare_by_Sex']].head()

,Sex,mean_Fare_by_Sex
0,male,25.523893
1,female,44.479818
2,female,44.479818
3,female,44.479818
4,male,25.523893


In [42]:
#性別ごとの港
df_train['count_Sex_x_Embarked'] = df_train.groupby(['Sex', 'Embarked'])['PassengerId'].transform('count')
df_train[['Sex', 'Embarked', 'count_Sex_x_Embarked']].head()

,Sex,Embarked,count_Sex_x_Embarked
0,male,S,441.0
1,female,C,73.0
2,female,S,203.0
3,female,S,203.0
4,male,S,441.0


In [41]:
#ハイパーパラメータの自動調整
import optuna

In [43]:
params_base = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : 'auc',
    'learning_rate' : 0.2,
    'num_leaves' : 16,
    'n_estimators' : 100000,
    'random_state' : 123,
    'importance_type' : 'gain',
    'bagging_freq' : 1,
    'seed' : 123,
}



#クロスバリデーション

def objective(trial):

    params_tuning = {
        'num_leaves' : trial.suggest_int('num_leaves', 8, 256),
        'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 5, 200),
        'min_sum_hession_in_leaf' : trial.suggest_float('min_sum_hession_in_leaf', 1e-5, 1e-2, log=True),
        'feature_fraction' : trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction' : trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'lambda_l1' : trial.suggest_float('lambda_l1', 1e-2, 1e2, log=True),
        'lambda_l2' : trial.suggest_float('lambda_l2', 1e-2, 1e2, log=True),
    }
    params_tuning.update(params_base)
    
    list_metrics = []
    imp = pd.DataFrame()

    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))

    for nfold in np.arange(5):
        idx_tr, idx_va = cv[nfold][0],cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr,:], y_train.loc[idx_tr,:]
        x_va, y_va = x_train.loc[idx_va,:], y_train.loc[idx_va,:]   

        model = lgb.LGBMClassifier(**params_tuning)
        model.fit(x_tr,
                  y_tr,
                  eval_set=[(x_tr, y_tr), (x_va, y_va)],
                  callbacks=[lgb.early_stopping(stopping_rounds=100)],
                 )
        
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = accuracy_score(y_va, np.where(y_va_pred>=0.5, 1, 0))
        list_metrics.append(metric_va)

    metrics = np.mean(list_metrics)

    return metrics

In [44]:
sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=30)

[I 2024-05-30 19:20:31,200] A new study created in memory with name: no-name-da8d76f4-ed79-407f-9316-c7ba7461a96f


[LightGBM] [Warning] Unknown parameter: min_sum_hession_in_leaf
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=0.492522233779106, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.492522233779106
[LightGBM] [Warning] seed is set=123, random_state=123 will be ignored. Current value: seed=123
[LightGBM] [Warning] lambda_l2 is set=83.76388146302445, reg_lambda=0.0 will be ignored. Current value: lambda_l2=83.76388146302445
[LightGBM] [Warning] bagging_fraction is set=0.8597344848927815, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8597344848927815
[LightGBM] [Warning] min_data_in_leaf is set=61, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=61
[LightGBM] [Warning] feature_fraction is set=0.7756573845414456, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7756573845414456
[LightGBM] [Warning] Unknown parameter: min_su

[I 2024-05-30 19:20:32,113] Trial 0 finished with value: 0.7059632163705982 and parameters: {'num_leaves': 181, 'min_data_in_leaf': 61, 'min_sum_hession_in_leaf': 4.792414358623587e-05, 'feature_fraction': 0.7756573845414456, 'bagging_fraction': 0.8597344848927815, 'lambda_l1': 0.492522233779106, 'lambda_l2': 83.76388146302445}. Best is trial 0 with value: 0.7059632163705982.


[LightGBM] [Warning] Unknown parameter: min_sum_hession_in_leaf
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=0.492522233779106, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.492522233779106
[LightGBM] [Warning] seed is set=123, random_state=123 will be ignored. Current value: seed=123
[LightGBM] [Warning] lambda_l2 is set=83.76388146302445, reg_lambda=0.0 will be ignored. Current value: lambda_l2=83.76388146302445
[LightGBM] [Warning] bagging_fraction is set=0.8597344848927815, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8597344848927815
[LightGBM] [Warning] min_data_in_leaf is set=61, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=61
[LightGBM] [Warning] feature_fraction is set=0.7756573845414456, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7756573845414456
[LightGBM] [Warning] Unknown parameter: min_su

[I 2024-05-30 19:20:32,655] Trial 1 finished with value: 0.7082041303119704 and parameters: {'num_leaves': 178, 'min_data_in_leaf': 99, 'min_sum_hession_in_leaf': 0.00015009027543233888, 'feature_fraction': 0.6715890080754348, 'bagging_fraction': 0.8645248536920208, 'lambda_l1': 0.567922374174008, 'lambda_l2': 0.01732652966363563}. Best is trial 1 with value: 0.7082041303119704.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[163]	training's auc: 0.774717	valid_1's auc: 0.72332
[LightGBM] [Warning] Unknown parameter: min_sum_hession_in_leaf
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=6.01372581918132, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.01372581918132
[LightGBM] [Warning] seed is set=123, random_state=123 will be ignored. Current value: seed=123
[LightGBM] [Warning] lambda_l2 is set=0.18101756169937222, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.18101756169937222
[LightGBM] [Warning] bagging_fraction is set=0.6817102653553969, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6817102653553969
[LightGBM] [Warning] min_data_in_leaf is set=102, min_child_samples=20 will be ig

[I 2024-05-30 19:20:45,307] Trial 28 finished with value: 0.6161634548992531 and parameters: {'num_leaves': 72, 'min_data_in_leaf': 184, 'min_sum_hession_in_leaf': 0.0006087327507705954, 'feature_fraction': 0.84731952840838, 'bagging_fraction': 0.6254957342799081, 'lambda_l1': 50.59746961823806, 'lambda_l2': 0.7308491614347951}. Best is trial 2 with value: 0.7126796811248509.


[LightGBM] [Warning] Unknown parameter: min_sum_hession_in_leaf
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=50.59746961823806, reg_alpha=0.0 will be ignored. Current value: lambda_l1=50.59746961823806
[LightGBM] [Warning] seed is set=123, random_state=123 will be ignored. Current value: seed=123
[LightGBM] [Warning] lambda_l2 is set=0.7308491614347951, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7308491614347951
[LightGBM] [Warning] bagging_fraction is set=0.6254957342799081, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6254957342799081
[LightGBM] [Warning] min_data_in_leaf is set=184, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=184
[LightGBM] [Warning] feature_fraction is set=0.84731952840838, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.84731952840838
[LightGBM] [Warning] Unknown parameter: min_su

[I 2024-05-30 19:20:45,575] Trial 29 finished with value: 0.698141987320319 and parameters: {'num_leaves': 38, 'min_data_in_leaf': 61, 'min_sum_hession_in_leaf': 0.000314095451385855, 'feature_fraction': 0.9250849383092855, 'bagging_fraction': 0.7831631986242769, 'lambda_l1': 12.244675589947384, 'lambda_l2': 0.1260073800737225}. Best is trial 2 with value: 0.7126796811248509.


[LightGBM] [Warning] Unknown parameter: min_sum_hession_in_leaf
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=12.244675589947384, reg_alpha=0.0 will be ignored. Current value: lambda_l1=12.244675589947384
[LightGBM] [Warning] seed is set=123, random_state=123 will be ignored. Current value: seed=123
[LightGBM] [Warning] lambda_l2 is set=0.1260073800737225, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1260073800737225
[LightGBM] [Warning] bagging_fraction is set=0.7831631986242769, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7831631986242769
[LightGBM] [Warning] min_data_in_leaf is set=61, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=61
[LightGBM] [Warning] feature_fraction is set=0.9250849383092855, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9250849383092855
[LightGBM] [Warning] Unknown parameter: mi

In [47]:
trial = study.best_trial
print('acc(best)={:.4f}'.format(trial.value))
display(trial.params)

acc(best)=0.7127


{'num_leaves': 107,
 'min_data_in_leaf': 149,
 'min_sum_hession_in_leaf': 3.52756635172055e-05,
 'feature_fraction': 0.5877258780737462,
 'bagging_fraction': 0.7657756869209191,
 'lambda_l1': 1.3406343673102123,
 'lambda_l2': 3.4482904089131434}

In [50]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

{'num_leaves': 16,
 'min_data_in_leaf': 149,
 'min_sum_hession_in_leaf': 3.52756635172055e-05,
 'feature_fraction': 0.5877258780737462,
 'bagging_fraction': 0.7657756869209191,
 'lambda_l1': 1.3406343673102123,
 'lambda_l2': 3.4482904089131434,
 'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'auc',
 'learning_rate': 0.2,
 'n_estimators': 100000,
 'random_state': 123,
 'importance_type': 'gain',
 'bagging_freq': 1,
 'seed': 123}